# WEEK_4_B_lda_infer_article
* Author: Jongyoon Kim & Eu-Bin KIM
* 23rd of August 2021
* tlrndk123@gmail.com

## 튜토리얼 목표
- 지난시간의 모든 과정을 이해했다면 직접 기사 하나를 준비해서, 기사의 토픽을 찾아봅시다.
- 어떻게 새로운 문서의 토픽도 추출할 수 있는 것일까요? 그 과정을 역설계 (reverse-engineer)해봅시다. 



# 지난시간

In [1]:
# 한국어 형태소 분석
!pip3 install konlpy
!pip3 install fsspec

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 65.0 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 118 kB 5.4 MB/s 


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from konlpy.tag import Kkma 
from tqdm import tqdm
from typing import List, Tuple
import requests
# Gensim
import gensim
import gensim.corpora as corpora
# Enable logging for gensim - optional
import logging
from sys import stdout
logging.basicConfig(stream=stdout, level=logging.INFO)
# 형태소 분석기로는 꼬꼬마 형태소 분석기를 사용합니다.
kkma = Kkma()

In [3]:
# 불용어 준비
# --- 한국어 불용어 불러오기 --- #
STOP_URL = "https://gist.githubusercontent.com/spikeekips/40eea22ef4a89f629abd87eed535ac6a/raw/4f7a635040442a995568270ac8156448f2d1f0cb/stopwords-ko.txt"
stop_words = requests.get(STOP_URL).text.split("\n")
 
# --- 한글 뉴스 데이터셋을 데이터프레임으로 불러오기 --- #
NEWS_URL = "https://gist.githubusercontent.com/ArtemisDicoTiar/7d91f779b2a9a7485009cb3f129fd711/raw/3a064717024895612f8684dbf8b7d67f5e70cae0/ko_news.csv"
news_df = pd.read_csv(NEWS_URL)\
            .drop(columns=['Unnamed: 0', 'f_name'])\
            .rename(columns={'idx': 'target', 'txt': 'content'})

In [4]:
# --- 데이터 전처리 --- #
# 리스트로 변경
docs = news_df.content.values.tolist()
# 이메일 제거
docs = [re.sub(r'\S*@\S*\s?', '', doc) for doc in docs] 
# 줄바꿈 기호 제거
docs = [re.sub(r'\s+', ' ', doc) for doc in docs]
# 따옴표 제거
docs = [re.sub(r"\'", "", doc) for doc in docs]
docs = [re.sub(r"“", "", doc) for doc in docs]
docs = [re.sub(r"”", "", doc) for doc in docs]
# 괄호 제거
docs = [re.sub(r"(\(\))", "", doc) for doc in docs]

In [5]:
# ---- 토크나이즈 & 품사 필터링 (토픽이 될 수 있는 품사만 가져오기) --- #
TARGET_TAGS = [
    'NNG',  # 일반 명사
    'NNP',  # 고유 명사
    'NNB',  # 의존 명사
    'NR',  # 수사
    'NP',  # 대명사
    'VV',  # 동사
    'VA',  # 형용사
    'MAG',  # 일반 부사
    'MAJ',  # 접속 부사
]
docs_tokenized: List[List[str]] = [
          [token for token, tag in kkma.pos(doc) if tag in TARGET_TAGS]
          for doc in tqdm(docs,"tokenizating & pos-filtering in progress...")
]

tokenizating & pos-filtering in progress...: 100%|██████████| 1600/1600 [20:08<00:00,  1.32it/s]


In [6]:
# ---- BoW 형식으로 변환하기 --- #
# 딕셔너리(단어 -> 정수, 정수 -> 단어) 구축
dictionary = corpora.Dictionary(docs_tokenized)
# 말뭉치 구축
corpus = [dictionary.doc2bow(tokens) for tokens in docs_tokenized]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(23181 unique tokens: ['11', '12', '13', '6', '가뜩이나']...) from 1600 documents (total 451332 corpus positions)


In [7]:
# --- LDA 모델 학습시작하기 --- #
# https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8,  # 찾고자하는 토픽의 개수
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

INFO:gensim.models.ldamodel:using autotuned alpha, starting with [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
INFO:gensim.models.ldamodel:using symmetric eta at 0.125
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 8 topics, 10 passes over the supplied corpus of 1600 documents, updating model once every 100 documents, evaluating perplexity every 1000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #100/1600
INFO:gensim.models.ldamodel:optimized alpha [0.21141398, 0.21892965, 0.20760126, 0.23408517, 0.21208921, 0.20718881, 0.21329436, 0.22976659]
INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #5 (0.207): 0.015*"것" + 0.014*"하" + 0.008*"회담" + 0.008*"김" + 0.007*"북한" + 0.006*"있" + 0.006*"고" + 0.006*"의원" + 0.006*"대하" + 0.005*"전"
INFO:gensi

In [8]:
# ---- 학습된 토픽 확인해보기 --- #
pprint(lda_model.print_topics())

INFO:gensim.models.ldamodel:topic #0 (0.250): 0.013*"연맹" + 0.012*"한국" + 0.010*"하" + 0.010*"협회" + 0.009*"등" + 0.009*"수" + 0.007*"대하" + 0.007*"선수권" + 0.007*"있" + 0.007*"카카오"
INFO:gensim.models.ldamodel:topic #1 (0.804): 0.028*"하" + 0.014*"것" + 0.012*"있" + 0.009*"되" + 0.008*"김" + 0.007*"수" + 0.006*"고" + 0.006*"없" + 0.006*"보" + 0.006*"나"
INFO:gensim.models.ldamodel:topic #2 (0.463): 0.016*"만" + 0.013*"스포츠" + 0.012*"1" + 0.012*"억" + 0.010*"2" + 0.009*"조" + 0.008*"3" + 0.008*"등" + 0.007*"회장" + 0.007*"5"
INFO:gensim.models.ldamodel:topic #3 (0.363): 0.016*"씨" + 0.009*"것" + 0.009*"등" + 0.009*"하" + 0.008*"김" + 0.008*"받" + 0.008*"대하" + 0.008*"경주" + 0.007*"경찰" + 0.006*"밝히"
INFO:gensim.models.ldamodel:topic #4 (0.460): 0.016*"것" + 0.012*"하" + 0.011*"김" + 0.008*"올림픽" + 0.008*"있" + 0.008*"북한" + 0.008*"대하" + 0.007*"대통령" + 0.007*"정상" + 0.007*"고"
INFO:gensim.models.ldamodel:topic #5 (0.576): 0.012*"것" + 0.010*"등" + 0.010*"있" + 0.009*"폰" + 0.009*"수" + 0.008*"게임" + 0.007*"만" + 0.005*"1" + 0.005*"스마트" + 0

# 이번시간
모든 뉴스기사를 바탕으로 문서-토픽 분포를 학습했으니,
새로운 뉴스문서가 들어와도 해당 뉴스의 토픽분포를 알아낼 수 있습니다. 

In [9]:
### TODO 1 ###
# 토픽을 찾고 싶은 기사의 본문을 복사해, article에 로드하세요!
article: str = """
외국계 회사 이사인 박모씨(44)는 백화점에 발길을 끊은 지 오래다. 박씨는 “백화점에 40~50대가 입을 만한 옷이 없어 모바일 앱 또는 온라인을 통해 쇼핑하고 있다”고 말했다.

불과 3년 전 백화점 4층 여성패션관에는 40~50대 여성이 자주 찾는 손정완·막스마라 등의 여성 브랜드가 즐비했다. 하지만 요즘 새로 문을 연 백화점에 중년 여성을 위한 브랜드는 찾아볼 수 없다. 기존 매장에서도 대부분 철수했다. 백화점들은 ‘에루샤’(에르메스·루이비통·샤넬) 등 명품 브랜드와 ‘준명품’으로 불리는 해외패션·컨템퍼러리 제품에 집중하고 있다. 무신사, 지그재그 등 20~30대 패션플랫폼 업체들이 최근 40~50대 여성 패션앱 시장에 앞다퉈 뛰어들고 있는 이유다.
무신사까지 가세한 4050세대 여성패션
붐비는 4050 패션앱…무신사도 뛰어든다
패션 플랫폼들은 40~50대 여성을 타깃으로 공격적으로 의류 카테고리를 넓혀가고 있다. 무신사는 올해 40~50대 여성을 위한 패션 플랫폼을 열 예정이다. 별도의 모바일 앱을 구축해 서비스를 출시할지, 무신사 앱 카테고리 안에 40~50대 여성패션관을 입점시킬지를 두고 조율 중이다. 무신사 관계자는 “상품 카테고리 확대 차원에서 시니어 여성 라인업을 준비하고 있다”고 설명했다. 무신사는 지난 6월 명품 카테고리를 개설해 상품군을 확대한 바 있다. 이번에는 연령층을 확대해 외연을 넓히겠다는 전략이다.

최근 들어 패션 플랫폼에 40~50대 여성을 위한 앱이 우후죽순으로 등장하고 있다. 20~30대 여성 패션 플랫폼인 지그재그는 중년 여성층을 겨냥해 지난 6월 ‘포스티’ 앱을 선보였으며, 40~50대 여성 커머스를 표방하는 ‘모라니크’는 7월 출시됐다. 백화점 또는 아울렛에서 철수한 중저가 브랜드를 대거 들여와 저렴한 가격에 판매하는 구조다. 롯데백화점에서 철수한 여성복 브랜드 ‘미세즈’ 등은 이런 패션 앱에서 80% 이상 저렴한 가격에 판매되고 있다. 패션 플랫폼 관계자는 “가장 큰 구매력을 지닌 40~50대 여성들이 최근 코로나19 확산으로 쇼핑할 곳이 마땅치 않아 비대면 플랫폼에 몰리고 있다”고 말했다. 중년 여성 플랫폼 퀸잇은 다운로드 수가 100만 명을 넘어섰다. 40~50대 빅사이즈 여성을 겨냥한 푸미의 다운로드 수도 10만 명을 넘어섰다.
"""
##############

In [10]:
def infer(article: str) -> List[Tuple[int, float]]:
  global lda_model, dictionary
  ### --- TODO 2 --- ###
  # article의 전처리를 진행하고, 토크나이즈를 하세요. (위 "지난시간"셀을 참고하기!)
  # dictionary.doc2bow()함수를 활용해, 토큰을 bow 형식으로 만들어 줍니다. 
  # https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow
  # 이후, lda_model의 get_document_topics() 함수를 활용해,해당 기사의 토픽분포를 예측해봅시다.
  # https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel.get_document_topics
  bow: List[Tuple[int, float]] = ... 
  topics: List[Tuple[int, float]] = ...
  return topics
  ###################### 

In [11]:
# 직접 예측된 기사의 토픽을 확인해보세요!
# 각 토픽 id의 단어분포가 궁금한 경우, lda_model.print_topic()을 사용해서 분포를 확인할 수 있습니다.
# https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel.print_topic
print(infer(article))

Ellipsis


## TODO 3 

어떻게 새로운 문서의 토픽도 추출할 수 있는 것일까요? 그 과정을 역설계 (reverse-engineer) 해보세요. BoW표현의 문서가 LDA의 입력으로 들어가면,
어떻게 토픽의 확률분포가 출력되는 것일까요? 수식을 사용하지 말고 설명해보세요!
